In [0]:
# !pip install tensorflow-gpu==2.0.0-alpha0


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, GRU, Dropout, Embedding
from music21 import converter, instrument, note, chord, pitch
from google.colab import drive
import numpy as np
import time
import glob
import pickle


 
Music21 v.4 is the last version that will support Python 2.
Please start using Python 3 instead.

Set music21.environment.UserSettings()['warnings'] = 0
to disable this message.



In [2]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
data_path = "/content/gdrive/My Drive/Applied Deep Learning/Data/"

In [0]:
def get_notes(filename):
  notes = []
  midi = converter.parse(filename)
  notes_to_parse = None
  try:  # file has instrument parts
    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse() 
  except:  # file has notes in a flat structure
    notes_to_parse = midi.flat.notes

  for element in notes_to_parse:
    
    new_chord = [-1] * 6 # 5 pitches + 1 duration
    
    if isinstance(element, note.Note):
      new_chord[0] = element.pitch.ps
      new_chord[5] = float(element.duration.quarterLength)
#       new_chord[5] = round(float(element.duration.quarterLength), 3)
      notes.append(new_chord)
    elif isinstance(element, chord.Chord):
      new_chord[5] = float(element.duration.quarterLength)
#       new_chord[5] = round(float(element.duration.quarterLength), 3)
      
      for k in range(len(element.pitches)):
        new_chord[k] = element.pitches[k].ps
        
        if k >= len(new_chord) - 1:
          break

      notes.append(new_chord)

  return notes


In [0]:
# pitch.Pitch(62)

In [18]:
from magenta.music import abc_parser as abc_parser
import numpy as np

MAX_NOTES = 2500
NUM_PARAM = 5 + 1 # 5 pitches + 1 duration

# 3D numpy array: num songs x MAX_NOTES x NUM_PARAM
all_songs_data = np.empty((1, MAX_NOTES, NUM_PARAM), np.float64)

for f in glob.glob(data_path + '*.mid'):
  song_notes = get_notes(f)

  temp_data = np.empty((1, 1, NUM_PARAM), np.float64)
  notes_data = np.zeros((1, MAX_NOTES, NUM_PARAM)) - 1
  
  for elem in song_notes:
    # Convert notes into a 2D numpy array
    # The 1st dimension is 5 pitches and 1 duration
    # The 2nd dimension is the number of notes
    raw_notes = np.array([[elem]]) 
    temp_data = np.append(temp_data, raw_notes, axis=1)
  
  temp_data = temp_data[:, 1:, :]
  notes_data[0, :temp_data.shape[1],:temp_data.shape[2]] = temp_data
#   print('notes_data = ', notes_data)
  
  all_songs_data = np.append(all_songs_data, notes_data, axis=0)

  
all_songs_data = all_songs_data[1:,:,:]
print(all_songs_data)


[[[52.         57.         -1.         -1.         -1.
    0.75      ]
  [40.         -1.         -1.         -1.         -1.
    0.75      ]
  [33.         40.         -1.         -1.         -1.
    0.75      ]
  ...
  [-1.         -1.         -1.         -1.         -1.
   -1.        ]
  [-1.         -1.         -1.         -1.         -1.
   -1.        ]
  [-1.         -1.         -1.         -1.         -1.
   -1.        ]]

 [[72.         -1.         -1.         -1.         -1.
    1.        ]
  [64.         -1.         -1.         -1.         -1.
    1.33333333]
  [36.         -1.         -1.         -1.         -1.
    2.        ]
  ...
  [-1.         -1.         -1.         -1.         -1.
   -1.        ]
  [-1.         -1.         -1.         -1.         -1.
   -1.        ]
  [-1.         -1.         -1.         -1.         -1.
   -1.        ]]

 [[-1.         -1.         -1.         -1.         -1.
   -1.        ]
  [-1.         -1.         -1.         -1.         -1.
   -1.

In [19]:
all_songs_data.shape

(92, 2500, 6)

In [0]:
import pickle

pickle.dump( all_songs_data, open( "all_songs_data.p", "wb" ) )

In [0]:
all_songs_data = pickle.load( open( "all_songs_data.p", "rb" ) )

# Convert to Music

In [21]:
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

from google.colab import files

import magenta.music as mm
import magenta
import tensorflow

print ('🎉 Done!')
print (magenta.__version__ )
print (tensorflow.__version__)

🎉 Done!
0.3.19
1.13.1


In [0]:
import numpy as np

In [0]:
# all_songs_data = all_songs_data[:,:100,:]

In [0]:
from magenta.protobuf import music_pb2

magenta_songs = []

for k in range(all_songs_data.shape[0]):
  song = music_pb2.NoteSequence()

  st_time = 0
  end_time = 0

  # Caution: Somehow the player cannot take a song that is longer than centain
  # number of notes. Here limiting to 200. Can adjust accordingly
  for m in range(200):
#   for m in range(all_songs_data.shape[1]):
    # Add the notes to the sequence.
    if all_songs_data[k,m,5] >= 0:
      st_time = end_time
      end_time = st_time + all_songs_data[k,m,5]

#       for n in range(1):
      for n in range(all_songs_data.shape[2]-1):
        pitch = int(all_songs_data[k,m,n])

        if pitch != -1:
          song.notes.add(pitch=pitch, 
                         start_time=st_time, 
                         end_time=end_time,
                         velocity=90)
  
  magenta_songs.append(song)


## !Warning: the MIDI player below seems to be working only in Python 2 

In [0]:
k = 0

# for k in range(len(magenta_songs)):
magenta_songs[k].total_time = 120
magenta_songs[k].tempos.add(qpm=100);
# This is a colab utility method that visualizes a NoteSequence.
mm.plot_sequence(magenta_songs[k])

# This is a colab utility method that plays a NoteSequence.
# mm.play_sequence(magenta_songs[k],synth=mm.fluidsynth)
mm.play_sequence(magenta_songs[k],synth=mm.synthesize)


Loading BokehJS ...